In [98]:
import numpy as np
import random
import os
import json

In [106]:
def createData(folder_name, N, m, M, numDay, day_to_remove_field = 1, num_day_to_keep_field = 1,
               num_field_to_stop = 2, num_field_to_harvests = None, day_to_stop = None):
    
    """
    # Arguments
        folder_name: A Keras model.
        N: Number of fields
        m: lower bound of number of field to harvest 
        M: upper bound of number of field to harvest 
        numDay: number of day to harvest of a feasible solution 
        day_to_remove_field = day to stop to harvest some beginning field 
        num_day_to_keep_field = number of consecutive day to the next time to remove fields
        num_field_to_stop = number of field to remove, 
        num_field_to_harvests = number of filed to harvest, 
        day_to_stop = day to remove number of field to harvest provided that M is still not surpass that number
            
    # Return
        Problem with feasible solutions
    """
    
    if M > num_field_to_harvests:
        raise ValueError('Value of `M` must not bigger than `num_field_to_harvests`')
        
    if not num_field_to_harvests:
        num_field_to_harvests = M + num_field_to_stop
    if not day_to_stop:
        day_to_stop = numDay # non-stop process
        
    print(N," ",m," ",M)
    dictionary = dict()
    dictionary['N'] = N
    dictionary['m'] = m
    dictionary['M'] = M
    
    d = [0] * N
    s = [0] * N
    e = [0] * N
    
    # add fields to harvest and remove these field from list of field that are not considered
    fields = [*range(N)]
    field_to_harvests = [*range(M)]
    del fields[:M]
    
    for day in range(numDay):
        # change `field_to_harvests`
        if day > day_to_remove_field and (day - day_to_remove_field) % num_day_to_keep_field == 0:
            field_to_harvests += fields[:num_field_to_stop]
            if len(field_to_harvests) > num_field_to_harvests + num_field_to_stop:
                del field_to_harvests[:num_field_to_stop]
            del fields[:num_field_to_stop]
            
        if day > day_to_stop:
            # start remove fields provided that the number of field to harvest is not smaller than M
            if len(field_to_harvests) > M + num_field_to_stop:
                del field_to_harvests[:num_field_to_stop] 
           
        # choose field to harvest
        num_harvested_fields = random.randint(m, M)
        harvested_fields = random.sample(field_to_harvests, num_harvested_fields)

        
        for field in harvested_fields:
            # one more time to be harvested
            d[field] += 1
            
            # set s to the first day to be harvested
            if (s[field] == 0):
                s[field] = day
                
            # e tracks the last day to be harvested
            e[field] = day
            
    dictionary['d'] = d
    dictionary['s'] = s
    dictionary['e'] = e
    
    arguments = dict()
    arguments['folder_name'] = folder_name
    arguments['N'] = N
    arguments['m'] = m
    arguments['M'] = M
    arguments['numDay'] = numDay
    arguments['day_to_remove_field'] = day_to_remove_field
    arguments['num_day_to_keep_field'] = num_day_to_keep_field
    arguments['num_field_to_stop'] = num_field_to_stop
    arguments['num_field_to_harvests'] = num_field_to_harvests
    arguments['day_to_stop'] = day_to_stop
    
    try:
        os.makedirs('./' + folder_name)
    except OSError as e:
        print(e)
    
    with open(folder_name + "/arguments.json", "w") as outfile:
        json.dump(arguments, outfile)
    
    with open(folder_name + "/data.json", "w") as outfile:
        json.dump(dictionary, outfile)

In [107]:
def readData(file_name):
    with open(file_name, 'r') as openfile:

        # Reading from json file
        json_object = json.load(openfile)
    return json_object

In [108]:
createData("N10_m2_M3/1", 10, 2, 3, numDay = 20, day_to_remove_field = 5, num_field_to_stop = 2, num_day_to_keep_field = 2,
           num_field_to_harvests = 6, day_to_stop = 15)

10   2   3


In [102]:
readData("sample.json")

{'N': 10,
 'm': 2,
 'M': 3,
 'd': [9, 7, 8, 6, 4, 4, 4, 3, 4, 2],
 's': [1, 1, 1, 7, 7, 11, 9, 14, 11, 17],
 'e': [10, 8, 13, 15, 16, 16, 19, 19, 18, 18]}